# Genera el árbol de todas las posiciones posibles de backgamon 

## Referencias
- https://en.wikipedia.org/wiki/Backgammon
- https://www.w3schools.com/python/
- https://www.sqlitetutorial.net/
- https://docs.python.org/3/library/sqlite3.html
- https://www.tutorialspoint.com/python_design_patterns/index.htm


In [23]:
_STANDARD_START_ = [
        2, 0, 0, 0, 0,-5,
        0,-3, 0, 0, 0, 5,
        -5, 0, 0, 0, 3, 0,
        5, 0, 0, 0, 0, 2
]
_ZERO_STATE_ = [
    0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0
]

START_POSITION = 0
END_POSITION = 23
MYSELF = 1
OPPONENT = -1
APPROACH_STAGE = 100
FINAL_RUN_STAGE = 200


class BackagmonState():



    _STANDARD_START_ = [
            2, 0, 0, 0, 0,-5,
            0,-3, 0, 0, 0, 5,
            -5, 0, 0, 0, 3, 0,
            5, 0, 0, 0, 0, -2
    ]
    _ZERO_STATE_ = [
        0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0
    ]

    START_POSITION = 0
    END_POSITION = 23
    MYSELF = 1
    OPPONENT = -1
    

    def __init__(self, tiles=_STANDARD_START_, home=[0,0], jail=[0,0]):
        self.tiles = tiles        
        self.home = home
        self.jail = jail
    
    def __str__(self):
        tiles =  [str(t) for t in self.tiles]         
        tiles = ",".join(tiles)
        home =  [str(h) for h in self.home]         
        home = ",".join(home)
        jail =  [str(j) for j in self.jail]         
        jail = ",".join(jail)
        return f"<{tiles}|{home}|{jail}>"

    def __repr__(self):        
        tiles =  [str(t) for t in self.tiles]         
        tiles = "".join(tiles)
        home =  [str(h) for h in self.home]         
        home = "".join(home)
        jail =  [str(j) for j in self.jail]         
        jail = "".join(jail)
        return f"{tiles}{home}{jail}"

        
    def game_stage(self, turn):

        sum = 0
        if turn == MYSELF:            
            for tile in self.tiles[0:17]:
                if tile > 0:
                    sum = sum + tile
        elif turn == OPPONENT:
            sum = 0
            for tile in self.tiles[6:23]:
                if tile < 0:
                    sum = sum + tile
        else: 
            raise Error(f'EL valor de turno es incorrecto. turno = {turno}')
        
        if sum == 0:
            return FINAL_RUN_STAGE
        else: 
            return APPROACH_STAGE


    def intentar_movimiento (self, ini_position, dice, turn):
        '''
        Intenta un movimiento a partir de:
            position: Posición inicial [0..23] desde la que va a intentar hacer el movimiento. 
            dice: valor de dado [1..6] indicando las posiciones que va a mover
            turn: turno al que le corresponde mover 1=yo, -1=oponente
        
        Devuelve: 
            objeto BackgammonState si puede realizar un movimiento válido 
            None si no puede realizar el movimiento
        '''

        end_position = ini_position + (dice * turn) #if turn = -1, it is opponent turn the move is in the opposite direction
        #ini_position_checkers = count_checkers(ini_position)
        #new_postion_checkers = count_checkers(end_position)

        movida = f'{self}<{ini_position},{dice},{turn}>'

        #Errores de software - violación de precondiciones - deberían logear Error y elevar una excepción
        sum_myself = 0
        sum_opponent = 0
        for tile in self.tiles:
            if tile > 0:
                sum_myself = sum_myself + tile
            elif tile < 0:
                sum_opponent = sum_opponent + tile        
        if sum_myself != 15 or sum_opponent != -15:
            print(f'{movida}: No es válido, el número de piezas es incorrecto')
            return None


        #Si la posición inicial debe estar siempre entre 0 y 23
        if ini_position < START_POSITION or ini_position > END_POSITION:
            print(f'{movida}: No es válido, dado que la posición inicial se sale de los límites')
            return None
        
        if dice < 1 or dice > 6:            
            print(f'{movida}: No es válido, dado que el valor del dado se sale de los límites')
            return None
        
                
        if turn != MYSELF and turn != OPPONENT:            
            print(f'{movida}: No es válido, dado que el turno no corresponde a ninguno de los oponentes')
            return None


        #Verificar el movimiento es válido - deberían logear un mensaje para verificar funcionamiento Ok 

        #Si estamos en la etapa de aproximación, la posición final debe estar dentro del tablero
        if estado.game_stage(turn)  == APPROACH_STAGE:
            if end_position < START_POSITION or end_position > END_POSITION:
                print(f'{movida}: No es válido, dado que la posición final se sale de los límites')
                return None
        #Si estamos en la etapa de carrera final, puede estar en los extremos y home
        elif estado.game_stage(turn)  == FINAL_RUN_STAGE:
            if turn == MYSELF and (end_position < 17 or end_position > END_POSITION + 1):
                print(f'{movida}: No es válido, dado que mi posición final se sale de los límites')
                return None
            elif turn == OPPONENT and (end_position > 5 or end_position < START_POSITION - 1):
                print(f'{movida}: No es válido, dado que la posición final del oponente se sale de los límites')
                return None
            else:
                print('{movida}: Algo salio muy mal. 0')
                return None
        else:
            print('Algo salió muy mal: 00')
            return None
                
        #Debe haber checkers > 0 si turn = 1 y checkers < 0 si turn = -1
        if (self.tiles[ini_position] * turn) <= 0: 
            print(f'{movida}: No es válido, dado que no hay fichas que mover en la posición')
            return None

        #Si esta llegando a home, debe estar en modo FINAL_RUN
        if turn == MYSELF and end_position == END_POSITION + 1 and self.game_stage(turn) != FINAL_RUN_STAGE:
            print(f'{movida}: Yo no puedo mover a home si no está en etapa FINAL_RUN_STAGE')
            return None
        if turn == OPPONENT and end_position == START_POSITION - 1 and self.game_stage(turn) != FINAL_RUN_STAGE:
            print(f'{movida}: Oponente no puede mover a home si no está en etapa FINAL_RUN_STAGE')
            return None
        


        print(f'{movida}: OK')
        

In [24]:
#Pruebas
estado = BackagmonState()
print(estado.__repr__()) #Kinjiru!!!!
print(estado) #llama automáticamente a __str__()
estado.intentar_movimiento(25, 2, 1)
estado.intentar_movimiento(22, 8, 1)
estado.intentar_movimiento(22, 1, 1)
estado.intentar_movimiento(0,1,1)
estado.intentar_movimiento(1,1,1)
estado.intentar_movimiento(1,1,-1)
estado.intentar_movimiento(0,1,-1)

estado = BackagmonState([-15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15])
print (estado.game_stage(1))
print (estado.game_stage(-1))


estado = BackagmonState([-12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2, 3,-3,5,0,5])
print (estado.game_stage(1))
print (estado.game_stage(-1))

estado = BackagmonState([-12,0,0,0,0,0,0,0,-4,0,0,-6,0,0,0,0,0,0,2, 3,-3,5,0,5])
estado.intentar_movimiento(25, 2, 1)
print (estado.game_stage(1))
print (estado.game_stage(-1))


estado = BackagmonState([-12,-1,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 3,2,5,0,5])
estado.intentar_movimiento(22, 2, 1)
estado.intentar_movimiento(1, 2, -1)



20000-50-30005-50003050000-20000
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0>
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0><25,2,1>: No es válido, dado que la posición inicial se sale de los límites
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0><22,8,1>: No es válido, dado que el valor del dado se sale de los límites
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0><22,1,1>: No es válido, dado que no hay fichas que mover en la posición
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0><0,1,1>: OK
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0><1,1,1>: No es válido, dado que no hay fichas que mover en la posición
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0><1,1,-1>: No es válido, dado que no hay fichas que mover en la posición
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2|0,0|0,0><0,1,-1>: No es válido, dado que la posición final se sale de los límites
200
200
200
100
<-12,0,0,0,